<a href="https://colab.research.google.com/github/vietcode/colab/blob/main/NZBGet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Settings

Run this section at least once. Changes will be saved automatically.



### Paths

In [ ]:
#@title {display-mode:"form", form-width: "50%", run: "auto"}

#@markdown #### Paths

#@markdown The main folder for all other folders.
main_dir = "/content/downloads" #@param {type:"string"}

#@markdown ---

#@markdown Temp folder, default to `{main_dir}/tmp`
temp_dir = "" #@param {type:"string"}
#@markdown Completed folder, default to `{main_dir}/completed`
dest_dir = "" #@param {type:"string"}
#@markdown Intermediate/cache folder, default to `{main_dir}/intermediate`
intermediate_dir = "" #@param {type:"string"}
#@markdown Queue folder, default to `{main_dir}/queue`
queue_dir = "" #@param {type:"string"}
#@markdown Folder for NZB files, default to `{main_dir}/nzb`
nzb_dir = "" #@param {type:"string"}

# Ensure MainDir exist.
!mkdir -p '{main_dir}'

if not temp_dir:
  temp_dir = main_dir + "/tmp"
if not dest_dir:
  dest_dir = main_dir + "/completed"
if not intermediate_dir:
  intermediate_dir = main_dir + "/intermediate"
if not queue_dir:
  queue_dir = main_dir + "/queue"
if not nzb_dir:
  nzb_dir = main_dir + "/nzb"

import os

# Install if haven't.
if not os.path.isdir("/content/.nzbget"):
  # Clean up some space.
  !rm -rf /content/sample_data/
  # Download setup script.
  !wget https://nzbget.net/download/nzbget-latest-bin-linux.run &> /dev/null
  # Install
  !sh nzbget-latest-bin-linux.run --destdir /content/.nzbget &> /dev/null
  !rm nzbget-latest-bin-linux.run
  # Symbolic link to PATH.
  !ln -s /content/.nzbget/nzbget /usr/local/bin

def nzbget(nzb_file, password = ""):
  # Store password (even if blank) into a temp file.
  !echo '{password}' > /content/.nzbget/passwords.txt

  if nzb_file:
    !nzbget \
      --option MainDir='{main_dir}' \
      --option TempDir='{temp_dir}' \
      --option DestDir='{dest_dir}' \
      --option InterDir='{intermediate_dir}' \
      --option QueueDir='{queue_dir}' \
      --option NzbDir='{nzb_dir}' \
      --option LockFile='{main_dir}/nzbget.lock' \
      --option LogFile='{main_dir}/nzbget.log' \
      --option Server1.Host='{server_host}' \
      --option Server1.Port='{server_port}' \
      --option Server1.Username='{server_user}' \
      --option Server1.Password='{server_pass}' \
      --option Server1.Encryption='{server_encryption}' \
      --option Server1.Cipher='{server_cipher}' \
      --option Server1.Connections='{server_connections}' \
      --option Server1.Retention='{server_retention}' \
      --option outputmode=color \
      --option ParCheck='auto' \
      --option ParScan='auto' \
      --option ParBuffer='128' \
      --option ParThreads='4' \
      --option UnpackPassFile='/content/.nzbget/passwords.txt' \
      --option ParPauseQueue='yes' \
      --option UnpackPauseQueue='yes' \
      --option DirectWrite='no' \
      --option WriteBuffer='1024' \
      '{nzb_file}'

  # Clean up stored password.
  !rm /content/.nzbget/passwords.txt

### Servers

In [ ]:
#@title {display-mode:"form", form-width: "50%", run: "auto"}
server_host = "news.newshosting.com" #@param {type:"string"}
server_port = 563 #@param {type:"integer"}
server_user = "" #@param {type:"string"}
server_pass = "" #@param {type:"string"}
server_encryption = "yes" #@param ["yes", "no"]
server_cipher = "" #@param {type:"string"}
server_connections = 10 #@param {type:"slider", min:5, max:60, step:1}
server_retention = 3650 #@param {type:"integer"}

## Downloads

Use a cell for each NZB file. Copy and paste existing cell to add more.

In [ ]:
#@title Download by NZB file already uploaded { display-mode: "form", form-width: "50%" }

nzb_file = "/content/We.Are.Freestyle.Love.Supreme.2020.2160p.HULU.WEB-DL.DDP5.1.HEVC-NTG (1).nzb" #@param {type:"string"}
password = "" #@param {type:"string"}

# Store password (even if blank) into a temp file.
!echo '{password}' > /content/.nzbget/passwords.txt

nzbget(nzb_file, password)

In [ ]:
#@title Click to upload NZB and start download { display-mode: "form", form-width: "50%" }

from google.colab import files

password = "" #@param {type:"string"}
uploaded = files.upload()

for name in uploaded.keys():
  content = uploaded[name]
  # Ensure NZB folder exists.
  !mkdir -p '{nzb_dir}'
  nzb_file = os.path.join(nzb_dir, name)
  # Write the uploaded NZB to filesystem.
  open(nzb_file, 'wb').write(content)
  # Start downloading
  nzbget(nzb_file, password)